## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.53,76,27,19.73,light rain
1,1,Mecca,SA,21.4267,39.8261,91.18,38,12,16.26,few clouds
2,2,Coari,BR,-4.0850,-63.1414,82.89,73,100,3.65,light rain
3,3,Galle,LK,6.0367,80.2170,82.00,77,27,2.48,scattered clouds
4,4,Taksimo,RU,56.3444,114.8794,24.84,70,87,3.04,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.53,76,27,19.73,light rain
6,6,Bredasdorp,ZA,-34.5322,20.0403,62.35,59,37,12.97,scattered clouds
11,11,Quartucciu,IT,39.2519,9.1794,71.02,44,0,21.99,clear sky
12,12,East London,ZA,-33.0153,27.9116,63.54,82,0,14.97,clear sky
14,14,Hilo,US,19.7297,-155.0900,75.27,94,100,4.61,moderate rain
18,18,Quelimane,MZ,-17.8786,36.8883,75.31,95,10,5.37,light rain
23,23,Nabire,ID,-3.3667,135.4833,76.44,91,80,2.64,broken clouds
24,24,Vaini,TO,-21.2000,-175.2000,69.96,100,75,0.00,broken clouds
26,26,Jamestown,US,42.0970,-79.2353,78.06,39,0,16.11,clear sky
27,27,Dingle,PH,10.9995,122.6711,77.14,89,93,6.98,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                254
City                   254
Country                249
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.53,light rain,-9.8000,-139.0333,
6,Bredasdorp,ZA,62.35,scattered clouds,-34.5322,20.0403,
11,Quartucciu,IT,71.02,clear sky,39.2519,9.1794,
12,East London,ZA,63.54,clear sky,-33.0153,27.9116,
14,Hilo,US,75.27,moderate rain,19.7297,-155.0900,
18,Quelimane,MZ,75.31,light rain,-17.8786,36.8883,
23,Nabire,ID,76.44,broken clouds,-3.3667,135.4833,
24,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000,
26,Jamestown,US,78.06,clear sky,42.0970,-79.2353,
27,Dingle,PH,77.14,overcast clouds,10.9995,122.6711,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)
        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.53,light rain,-9.8000,-139.0333,Villa Enata
6,Bredasdorp,ZA,62.35,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor
11,Quartucciu,IT,71.02,clear sky,39.2519,9.1794,Best Western Hotel Residence Italia
12,East London,ZA,63.54,clear sky,-33.0153,27.9116,Tu Casa
14,Hilo,US,75.27,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
18,Quelimane,MZ,75.31,light rain,-17.8786,36.8883,Hotel Flamingo
23,Nabire,ID,76.44,broken clouds,-3.3667,135.4833,Hotel Nusantara I
24,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
26,Jamestown,US,78.06,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
27,Dingle,PH,77.14,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.53,light rain,-9.8000,-139.0333,Villa Enata
6,Bredasdorp,ZA,62.35,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor
11,Quartucciu,IT,71.02,clear sky,39.2519,9.1794,Best Western Hotel Residence Italia
12,East London,ZA,63.54,clear sky,-33.0153,27.9116,Tu Casa
14,Hilo,US,75.27,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
18,Quelimane,MZ,75.31,light rain,-17.8786,36.8883,Hotel Flamingo
23,Nabire,ID,76.44,broken clouds,-3.3667,135.4833,Hotel Nusantara I
24,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
26,Jamestown,US,78.06,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
27,Dingle,PH,77.14,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY


In [13]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))